In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
import pandas as pd

In [2]:
data = pd.read_csv("Data/depression_llm_filtered_en.csv")

In [3]:
pd.set_option('display.max_colwidth', None)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3385 entries, 0 to 3384
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  3385 non-null   object
 1   Text    3380 non-null   object
dtypes: object(2)
memory usage: 53.0+ KB


In [5]:
data.head()

,Result,Text
0,yes,"Mentally, I feel I'm on the verge of a breakdown. I can't connect to my family, and have no one to turns to for understanding and companionship."
1,yes,"that tenuous grasp you had on climbing out of the hole, that little bit of hope that things are starting to turn around, suddenly releases and you are plummet back into the pit; and now you are too damn tired to try climbing out again"
2,yes,depression hits you and you wish you could get back to sleep
3,yes,I feel so ragged and unclean in comparison. It's not that I feel like they hate me or are judging me. But I feel like deep down they know there's something wrong with me.
4,yes,I feel like I don’t have a mom to hang with or comfort me. I’ve been seriously depressed


In [6]:
data = data[data['Text'].apply(lambda x: isinstance(x, str))]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3380 entries, 0 to 3384
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Result  3380 non-null   object
 1   Text    3380 non-null   object
dtypes: object(2)
memory usage: 79.2+ KB


In [8]:
data.head()

,Result,Text
0,yes,"Mentally, I feel I'm on the verge of a breakdown. I can't connect to my family, and have no one to turns to for understanding and companionship."
1,yes,"that tenuous grasp you had on climbing out of the hole, that little bit of hope that things are starting to turn around, suddenly releases and you are plummet back into the pit; and now you are too damn tired to try climbing out again"
2,yes,depression hits you and you wish you could get back to sleep
3,yes,I feel so ragged and unclean in comparison. It's not that I feel like they hate me or are judging me. But I feel like deep down they know there's something wrong with me.
4,yes,I feel like I don’t have a mom to hang with or comfort me. I’ve been seriously depressed


from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)

In [9]:
from transformers import MarianMTModel, MarianTokenizer
import torch
from tqdm import tqdm  # Import tqdm for the progress bar

model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def batch_translate_text(texts, batch_size=8):
    translated_texts = []
    # Use tqdm to create a progress bar
    for i in tqdm(range(0, len(texts), batch_size), desc="Translating"):
        batch_texts = texts[i:i + batch_size]
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True).to(device)
        translated = model.generate(**inputs)
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
        translated_texts.extend(translated_batch)
    return translated_texts

# Example data
# data = pd.DataFrame({'Text': ['Text 1', 'Text 2', 'Text 3', ...]})
texts = data['Text'].tolist()
translated_texts = batch_translate_text(texts)

data['Translated_Text'] = translated_texts

print(data)


C:\Users\ensar\Desktop\dogaldilisleme\lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Translating: 100%|█████████████████████████████████████████████████████████████████| 423/423 [2:11:02<00:00, 18.59s/it]

     Result  \
0       yes   
1       yes   
2       yes   
3       yes   
4       yes   
...     ...   
3380    yes   
3381    yes   
3382    yes   
3383    yes   
3384    yes   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                        Text  \
0                                                                                                                                                                                                                                                                                                                           Mentally, I feel I'm on the verge of a 

In [10]:
data

,Result,Text,Translated_Text
0,yes,"Mentally, I feel I'm on the verge of a breakdown. I can't connect to my family, and have no one to turns to for understanding and companionship.","Zihinsel olarak, bir çöküşün eşiğinde olduğumu hissediyorum. Ailemle bağlantı kuramıyorum ve anlayış ve arkadaşlık için başvuracak kimsem yok."
1,yes,"that tenuous grasp you had on climbing out of the hole, that little bit of hope that things are starting to turn around, suddenly releases and you are plummet back into the pit; and now you are too damn tired to try climbing out again","Delikten tırmanmak için sahip olduğunuz o ince kavrama, işlerin dönmeye başladığı, aniden serbest kaldığı ve çukura geri düştüğüne dair küçük bir umut; ve şimdi tekrar tırmanmayı denemek için çok yorgunsun."
2,yes,depression hits you and you wish you could get back to sleep,Depresyon sana çarpar ve uykuya geri dönebilmeyi dilersin
3,yes,I feel so ragged and unclean in comparison. It's not that I feel like they hate me or are judging me. But I feel like deep down they know there's something wrong with me.,Kıyaslamada kendimi çok huysuz ve kirli hissediyorum. Benden nefret ettiklerini ya da beni yargıladıklarını hissetmiyorum. Ama içimde bir sorun olduğunu biliyorlar gibi hissediyorum.
4,yes,I feel like I don’t have a mom to hang with or comfort me. I’ve been seriously depressed,Takılacak ya da beni rahatlatacak bir annem yokmuş gibi hissediyorum. Ciddi şekilde depresyondayım.
...,...,...,...
3380,yes,After years of struggling with depression and anxiety I've noticed I can't concentrate and process information as fast as I used to.,"Yıllarca depresyon ve anksiyete ile mücadele ettikten sonra, bilgileri eskisi kadar hızlı konsantre edemediğimi ve işleyemediğimi fark ettim."
3381,yes,"I hate my mental illness. I hate that my coping mechanism of choice is self harm... My mental illness sucks and my self harm might be ugly... I might have 5,000 reasons that I think I should die but I will sit here and provide anyone I can with 5,000 reasons they shouldn't die... The reminder that people like this exist doesn't help with my mental health issues and only provides me with less hope for the future",Zihinsel hastalığımdan nefret ediyorum. Seçim mekanizmamın kendi kendine zarar vermesinden nefret ediyorum... Zihinsel hastalığım berbat ve kendi zararım çirkin olabilir... Ölmem gerektiğini düşündüğüm 5.000 nedenim olabilir ama burada oturup ölmemeleri için 5.000 nedenim olan herkese yardım edeceğim... Böyle insanların var olmasının hatırlatıcısı zihinsel sağlık sorunlarıma yardımcı olmaz ve sadece bana gelecek için daha az umut verir.
3382,yes,a burning feeling inside of me that my life will end on my own terms,Hayatımın kendi şartlarımla sona ereceğini içimde yanan bir his
3383,yes,"I have had an urge to slit my throat and I don't know why...I want to plunge it into my heart and leave this planet...I'm in constant pain mentally, and I just want it to stop...I hate myself, others, and the idea of existence...Deep down, I know i'm probably gonna end my life soon by losing control of myself and going off the deep end.","Boğazımı kesmek için bir dürtüm vardı ve nedenini bilmiyorum... Kalbime dalıp bu gezegeni terk etmek istiyorum... Zihinsel olarak sürekli acı çekiyorum ve sadece durmasını istiyorum... Kendimden, diğerlerinden ve varoluş fikrinden nefret ediyorum... Derinlerde, muhtemelen hayatımı yakında kendimin kontrolünü kaybederek ve derinlerden ayrılarak sona erdireceğimi biliyorum."


In [11]:
data.to_excel("depression_reddit_translated.xlsx", index=False)